In [1]:
import os

In [2]:
ANNOTATIONS_PATH = "./annotations/instances_train2017.json"

!pip install tqdm
if not os.path.exists(ANNOTATIONS_PATH):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip annotations_trainval2017.zip

In [3]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios


anchors_ratios = get_optimal_anchors_ratios(
    ANNOTATIONS_PATH,
    input_size=512,
    scale_bboxes=True,
    num_runs=10,
    num_anchors_ratios=3,
    max_iter=300,
    min_size=0,
    decimals=1,
)

2020-05-23 09:04:27 Starting the calculation of the optimal anchors ratios
2020-05-23 09:04:27 Reading annotations from ./annotations/instances_train2017.json
2020-05-23 09:04:45 Extracting and preprocessing bounding boxes
2020-05-23 09:04:55 Discarding 2 bounding boxes with size lower or equal to 0
2020-05-23 09:04:55 K-Means (10 runs): 100%|████████████| 10/10 [01:42<00:00, 10.20s/it]2020-05-23 09:06:37 Runs avg. IoU: 80.48% ± 0.00% (mean ± std. dev. of 10 runs, 0 skipped)
2020-05-23 09:06:37 Optimal anchors ratios (avg. IoU: 80.48%): [(0.6, 1.5), (1.0, 1.0), (1.4, 0.7)]



In [4]:
anchors_ratios

[(0.6, 1.5), (1.0, 1.0), (1.4, 0.7)]